In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Loading

In [3]:
# from git import Repo
from langchain.text_splitter import Language
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser

# Clone
# repo_path = "C:\\Users\\admin\\bin\code\\book-management-system"
repo_path = "/Users/i353667/Documents/code/github.com/book-management-system"
# repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".java"],
    exclude=["**/target/**"],
    parser=LanguageParser(language=Language.JAVA, parser_threshold=500),
)
documents = loader.load()
len(documents)

6

## Splitting

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JAVA, chunk_size=2000, chunk_overlap=200
)
texts = python_splitter.split_documents(documents)
len(texts)

6

## RetrievalQA

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

## Chat

In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

question = "Explains BookController.java"
result = qa.invoke(question)
result["answer"]

/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6
/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use 

'In BookController.java, there is a REST controller class named "BookController" that handles HTTP requests related to books. It has two methods:\n\n1. `addBook`: This method is used to add a new book to the repository. It takes a BookDTO object as a request body and checks if the book already exists in the repository based on its ISBN. If the book does not exist, a new Book object is created and added to the repository. If the book already exists, the quantity in stock of the existing book is increased by 1. The method returns a ResponseEntity with a success message.\n\n2. `searchBooks`: This method is used to search for a book in the repository based on its ISBN. It takes the ISBN as a path variable and retrieves the book from the repository. If the book is not found, it returns a ResponseEntity with a NOT_FOUND status. If the book is found, it returns a ResponseEntity with the book object.\n\nThere is also another method named `getAllBooks` that is used to retrieve all books from th

'BookController.java is a Spring MVC controller class that handles incoming HTTP requests related to books. It contains two main methods:
1. addBook(): This method is mapped to the POST request /book. It takes a BookDTO object as a request body and adds a book to the repository. It first checks if a book with the same ISBN already exists in the repository. If it does not exist, a new Book object is created and populated with the information from the BookDTO object. The quantityInStock property is set to 1. The new book is then added to the repository using the BookService class. If a book with the same ISBN already exists, the quantityInStock property of the existing book is increased by 1 and the updated book is added to the repository.
2. searchBooks(): This method is mapped to the GET request /search/{isbn}, where {isbn} is a path variable representing the ISBN of the book to search for. It retrieves a book from the repository based on its ISBN. If the book is found, it is returned in the response body with a status code of 200 (OK). If the book is not found, a 404 (NOT_FOUND) status code is returned.
3. getAllBooks(): This method is mapped to the GET request /search. It retrieves all books from the repository and returns them in the response body with a status code of 200 (OK).
The BookController class is annotated with @RestController, indicating that it is a controller class that handles RESTful requests and automatically serializes the response to JSON. It is also annotated with @RequestMapping, specifying the base URL mapping for all the methods in the class. The BookService class is injected into the controller using the @Autowired annotation, allowing the controller to use its methods to interact with the repository.'

In [1]:
question = "give me integration test Java code for BookController, only code, don't append other message."
result = qa.invoke(question)
result["answer"]

NameError: name 'qa' is not defined

'The integration test Java code for BookController would typically involve using a testing framework like JUnit and mocking dependencies using tools like Mockito. Here\'s an example of how the integration test code for BookController might look like:
```java
@RunWith(SpringRunner.class)
@SpringBootTest(webEnvironment = SpringBootTest.WebEnvironment.RANDOM_PORT)
@AutoConfigureMockMvc
public class BookControllerIntegrationTest {
    @Autowired
    private MockMvc mockMvc;
    @MockBean
    private BookService bookService;
    @Autowired
    private ObjectMapper objectMapper;
    @Test
    public void testAddBook() throws Exception {
        // Create a BookDTO object with test data
        BookDTO bookDTO = new BookDTO();
        bookDTO.setTitle("Test Book");
        bookDTO.setAuthor("Test Author");
        bookDTO.setIsbn("1234567890");
        bookDTO.setPublisher("Test Publisher");
        // Convert the BookDTO object to JSON
        String json = objectMapper.writeValueAsString(bookDTO);
        // Mock the behavior of the bookService
        when(bookService.getBookByIsbn(anyString())).thenReturn(null);
        // Perform the POST request to add a book
        mockMvc.perform(post("/book")
                .contentType(MediaType.APPLICATION_JSON)
                .content(json))
                .andExpect(status().isOk())
                .andExpect(content().string("New Book added."));
        // Verify that the bookService method was called
        verify(bookService, times(1)).addBook(any(Book.class));
    }
    @Test
    public void testSearchBooks() throws Exception {
        // Create a Book object with test data
        Book book = new Book();
        book.setTitle("Test Book");
        book.setAuthor("Test Author");
        book.setIsbn("1234567890");
        book.setPublisher("Test Publisher");
        // Mock the behavior of the bookService
        when(bookService.getBookByIsbn("1234567890")).thenReturn(book);
        // Perform the GET request to search for a book
        mockMvc.perform(get("/search/1234567890"))
                .andExpect(status().isOk())
                .andExpect(jsonPath("$.title", is("Test Book")))
                .andExpect(jsonPath("$.author", is("Test Author")))
                .andExpect(jsonPath("$.isbn", is("1234567890")))
                .andExpect(jsonPath("$.publisher", is("Test Publisher")));
    }
    @Test
    public void testGetAllBooks() throws Exception {
        // Create a list of Book objects with test data
        List<Book> books = new ArrayList<>();
        Book book1 = new Book();
        book1.setTitle("Test Book 1");
        book1.setAuthor("Test Author 1");
        book1.setIsbn("1234567890");
        book1.setPublisher("Test Publisher 1");
        books.add(book1);
        Book book2 = new Book();
        book2.setTitle("Test Book 2");
        book2.setAuthor("Test Author 2");
        book2.setIsbn("0987654321");
        book2.setPublisher("Test Publisher 2");
        books.add(book2);
        // Mock the behavior of the bookService
        when(bookService.getAllBooks()).thenReturn(books);
        // Perform the GET request to get all books
        mockMvc.perform(get("/search"))
                .andExpect(status().isOk())
                .andExpect(jsonPath("$[0].title", is("Test Book 1")))
                .andExpect(jsonPath("$[0].author", is("Test Author 1")))
                .andExpect(jsonPath("$[0].isbn", is("1234567890")))
                .andExpect(jsonPath("$[0].publisher", is("Test Publisher 1")))
                .andExpect(jsonPath("$[1].title", is("Test Book 2")))
                .andExpect(jsonPath("$[1].author", is("Test Author 2")))
                .andExpect(jsonPath("$[1].isbn", is("0987654321")))
                .andExpect(jsonPath("$[1].publisher", is("Test Publisher 2")));
    }
}
```
Note that this is just an example and you may need to modify it based on your specific requirements and dependencies.'